In [6]:
import pandas as pd
import os
import re
import random
import asyncio
from playwright.async_api import Locator
import os
import glob
import pandas as pd


In [2]:
from playwright.async_api import Playwright, async_playwright
playwright = await async_playwright().start()

In [3]:

async def login():
    browser = await playwright.chromium.launch(headless=False)
    context = await browser.new_context()
    page = await context.new_page()
    await page.goto("https://marketing.toridori.me/campaigns/85296/campaign_entries")
    await page.goto("https://marketing.toridori.me/signin")
    await page.get_by_label("email").fill("mike.ogawa@techmo.jp")
    await asyncio.sleep(random.randint(3,5))
    await page.get_by_test_id("password").fill("1234567")
    await asyncio.sleep(random.randint(3,5))
    await page.get_by_label("ログイン").click()
    await asyncio.sleep(random.randint(3,6))
    return page
    
page = await login()

In [4]:
async def access_to_campaign():
    await page.goto("https://marketing.toridori.me/campaigns/85296/campaign_entries")
    await page.get_by_role("menuitem", name="キャンペーン一覧").nth(1).click()
    await asyncio.sleep(random.randint(1,2))
    await page.locator("a.MuiCardActionArea-root").click()
    await asyncio.sleep(random.randint(1,2))
    await page.locator("a.MuiButtonGroup-groupedHorizontal").nth(2).click()
    await asyncio.sleep(random.randint(1,5))
    return page

page = await access_to_campaign()

In [7]:
PREVIOUS_DFs = sorted(glob.glob("post_*/**"))
N = len(PREVIOUS_DFs) + 1
RESULT_DIR = f"result_{N:02d}"

print(PREVIOUS_DFs)
previous_df=pd.concat([pd.read_csv(df_f) for df_f in PREVIOUS_DFs], ignore_index=True)
previous_done_names = previous_df["name"].to_list()

def is_already_done_by_name(name: str) -> bool:
    return name in previous_done_names

async def loop_per_table(page: Locator, res_list: list[dict[str, str]]) -> tuple[Locator, bool, bool]:
    res_list = []
    should_continue = True
    for tr in await page.locator("tbody tr").all():
        await tr.click()
        await asyncio.sleep(random.randint(3,5))
        page, res = await on_per_page(page)
        # print(res["name"])
        if is_already_done_by_name(res["name"]):
            print(f"{res['name']} is already done")
            should_continue = False
            break
        await asyncio.sleep(random.randint(1,2))
        res_list.append(res)
        # if len(res_list) > 2:break
    return page, res_list, should_continue

async def next_table(page: Locator) -> tuple[Locator, bool]:
    el = page.locator("button.MuiPaginationItem-root").nth(-1)
    disabled = await el.is_disabled()
    if not disabled:
        await el.click()
        await asyncio.sleep(random.randint(2,4))
    return page, disabled

async def on_per_page(locator: Locator) -> tuple[Locator, dict[str, str]]:
    drawer_el = page.locator(".MuiPaper-root.MuiPaper-elevation.MuiDrawer-paperAnchorDockedRight")
    close_icon = drawer_el.locator(".MuiBox-root svg").nth(0)
    # print("svg_icons done")
    res = {}
    try:

        expiration_div = drawer_el.get_by_text(re.compile("保留期間")).locator("..").nth(0)
        svgs = await expiration_div.locator("svg").all()
        if svgs:
            aria_label = await svgs[0].get_attribute("aria-label")
            res["aria_label"] = aria_label
        else:
            res["aria_label"] = await expiration_div.locator("span").nth(0).inner_text()
        # print("aria_label done")
        sns_box = drawer_el.get_by_text("SNS連携").locator("..")
        sns_link = await sns_box.locator("a").get_attribute("href")
        followers = await sns_box.locator("span.MuiTypography-caption").inner_text()
        icon_html = await sns_box.locator(".MuiListItemButton-root svg").nth(0).evaluate("e => e.outerHTML")
        res["sns_link"] = sns_link
        res["followers"] = followers
        res["icon_html"] = icon_html
        # print("followers done")
        # print("icon_html done")
        name = await drawer_el.locator(".MuiTypography-root.MuiTypography-h6").inner_text()
        res["name"] = name
        # print("name done")

        desc = await drawer_el.locator(".MuiBox-root > .MuiBox-root > .MuiBox-root > p.MuiTypography-root.MuiTypography-body2").nth(-1).inner_text()
        res["desc"] = desc

        title_el = drawer_el.locator(".MuiTypography-root.MuiTypography-h6").locator("..")
        gender_icon_label = await title_el.locator("svg").nth(0).get_attribute("data-testid")
        res["gender_icon_label"] = gender_icon_label
        # print("gender_icon_label done")
        
        age = await drawer_el.locator(".MuiTypography-root.MuiTypography-body2").nth(1).inner_text()
        res["age"] = age

        img_src = await drawer_el.locator(".MuiAvatar-circular img").get_attribute("src")
        res["img_src"] = img_src
        # print("age done")
        
        if not await page.get_by_text("Instagramのハッシュタグ傾向").all():
            raise Exception("Instagramのハッシュタグ傾向が見つかりません")
        
        word_cloud_img_src = await (page
                        .get_by_text("Instagramのハッシュタグ傾向")
                        .locator("..")
                        .locator("img")
                        .get_attribute("src")
                    )
        res["word_cloud_img_src"] = word_cloud_img_src

        infos_1 = await drawer_el.locator(".MuiGrid-root.MuiGrid-item.MuiGrid-grid-xs-1\.5").all()
        infos_2 = await drawer_el.locator(".MuiGrid-root.MuiGrid-item.MuiGrid-grid-xs-2").all()
        comments = await infos_1[0].inner_text()
        engagement = await infos_1[1].inner_text()
        likes = await infos_2[0].inner_text()
        res["comments"] = comments
        res["engagement"] = engagement
        res["likes"] = likes

        followers_gender = await drawer_el.locator(".recharts-default-legend span[style='font-size: 1rem; font-weight: bold;']").all()
        followers_gender = [await f.inner_text() for f in followers_gender]

        follower_list = []
        for follower_gender in followers_gender:
            res_num = re.findall("[0-9]+", follower_gender)
            follower_list.append(res_num[0]) if res_num else follower_list.append("0")          
        
        for v, key in zip(follower_list, [
            "female_follower", 
            "male_follower", 
            "unknown_follower"]):
            res[key] = v

    except Exception as e:
        print(e)

    finally:
        await close_icon.click()
    return page, res   


def save_to_csv(i:int, res_list: list[dict[str, str]]) -> None:
    if not res_list:
        return
    df = pd.DataFrame(res_list)
    os.makedirs(RESULT_DIR, exist_ok=True)
    df.to_csv(os.path.join(RESULT_DIR, f"res_{i:03d}.csv"), index=False)

['post_result_01/post_result_01.csv', 'post_result_02/post_result.csv']


In [8]:
MAX_PAGES = 30

for i in range(1, MAX_PAGES):
    res_min = []
    page, res_min, should_continue = await loop_per_table(page, res_min)
    save_to_csv(i, res_min)
    page, is_disabled = await next_table(page)
    await asyncio.sleep(random.randint(3, 7))
    if is_disabled or not should_continue:
        break


Instagramのハッシュタグ傾向が見つかりません
Instagramのハッシュタグ傾向が見つかりません
Instagramのハッシュタグ傾向が見つかりません
Instagramのハッシュタグ傾向が見つかりません
Instagramのハッシュタグ傾向が見つかりません
Instagramのハッシュタグ傾向が見つかりません
Instagramのハッシュタグ傾向が見つかりません
Instagramのハッシュタグ傾向が見つかりません
Instagramのハッシュタグ傾向が見つかりません
Instagramのハッシュタグ傾向が見つかりません
ゆう is already done


In [9]:
await page.close()
await playwright.stop()

In [11]:
from glob import glob
FROM_PATH=f"result_{N:02d}/"
TO_PATH=f"post_result_{N:02d}/"
FNAME = "post_result.csv"
columns_order = [
    'aria_label',
    'followers',
    'icon_html',
    'name',
    'gender_icon_label',
    'age',
    'img_src',
    'word_cloud_img_src',
    'desc',
    'comments',
    'engagement',
    'likes',
    'female_follower',
    'male_follower',
    'unknown_follower',
    'sns_link',
]

df=pd.concat([pd.read_csv(x) for x in sorted(glob(os.path.join(FROM_PATH,"*.csv")))], ignore_index=True)
df = (
        pd.merge(df, previous_df["name"], on="name", indicator=True, how='outer')
        .query('_merge=="left_only"')
        .drop('_merge', axis=1)
    )
df = df[columns_order]
os.makedirs(TO_PATH, exist_ok=True)
df.to_csv(os.path.join(TO_PATH, FNAME), index=False)